<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/wordmark_black65ee464.png" width="700">

# Week 2 : Final Challenge

**Welcome to the final challenge!**  

In the previous notebook we've seen how we can use the VQC class in Aqua to classify the digits `0` and `1`. However, classifying `0` and `1` is relatively simple as digits `0` and `1` are easily distinguishable. `4` and `9` however, are notoriously similar, with a _loop_ on the top and a _line_ on the bottom. This can be corroborated looking at our 2-D t-SNE plot from the previous notebook (Fig.2), we see that `0` and `1` are clustered relatively far from each other making them easily distinguishable, however `4` and `9` are overlapping. In this challenge we are providing you with a dataset with digits reduced to **dimension 3**. For example, in Fig.1 we can see the dimension reduction of the 784 dimension vector for digit `4` into a dimension 3 feature vector. 

**Fig.1 : Features of the digit `4` after reducing dimension to 3:** 
<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/four2a7701f.png" width="700">

**Fig.2 : MNIST dataset after dimension reduction to 2 as given in the previous notebook:**
<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/mnist_plot53adb39.png" width="400">

## Challenge Question   
Use the VQC method from Aqua to classify the digits `4` and `9` as given in the dataset **challenge_dataset_4_9_dim3_training.csv** provided to you. 

## Rules and Guidelines

* Your `QuantumCircuit` can have a **maximum of 6 qubits**.
* **Cost of the circuit should be less than 2000**.  
* You should not change names of the functions `feature_map()` , `variational_circuit()`  and `return_optimal_params()`.
* All the functions must return the value types as mentioned. 
* All circuits must be Qiskit generated.
* Best of all submissions is considered for grading.

## Judging criteria 

* Primary judgement is based on the **accuracy of the model**, higher the better. **Accuracies which differ by less than 0.005 will be considered to be equal**. ex: Accuracies 0.7783 and 0.7741 will be considered to be equal.
* If the accuracies are tied, the tie will be broken using **cost of the circuit** as the metric, lower the better. 
* In the case that both accuracy of the model and cost of the circuit are equal, **time of submission** is taken into account, Earlier the better. 

_**Important Note:**_ The **leaderboard shown during the progress of the competition** will only display accuracy of the model and is **not the final leaderboard**. Breaking ties between accuracy of the model by considering lower **cost of circuit** will only be done after the competition ends. **The final leaderboard will be announced post the event** which will take into consideration cost of the circuit and time of submission. 

## Certificate Eligibility

Everyone who scores an **accuracy greater than 0.70 (i.e, 70%) will be eligible for a certificate**. 


An explanation on how to calculate the accuracy of the model and the cost of the circuit is given in the end inside the `grade()` function. Before you submit, make sure the grading function is running on your device. To save time you can also use the grading function provided to calculate the accuracy and circuit cost without having to submit your solution onto HackerEarth. Remember, your final score will be determined using the same grading methods as given in this notebook, but will be evaluated on unseen datapoints.

In [1]:
# installing a few dependencies
# !pip install --upgrade seaborn==0.10.1
# !pip install --upgrade scikit-learn==0.23.1
# !pip install --upgrade matplotlib==3.2.0
# !pip install --upgrade pandas==1.0.4
# !pip install --upgrade qiskit==0.19.6 
# !pip install --upgrade plotly==4.9.0

# the output will be cleared after installation
from IPython.display import clear_output
clear_output()

In [1]:
# we have imported a few libraries we thing might be useful 
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer

from qiskit import *
import numpy as np
from qiskit.visualization import plot_bloch_multivector, plot_histogram
%matplotlib inline
import matplotlib.pyplot as plt

import time
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap, RealAmplitudes, EfficientSU2
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.optimizers import COBYLA


# The the write_and_run() magic function creates a file with the content inside the cell that it is run. 
# You have used this in previous exercises for creating your submission files. 
# It will be used for the same purpose here.

from IPython.core.magic import register_cell_magic
@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

# Solution

## Data loading 

This notebook has helper functions and code snippets to save your time and help you concentrate on what's important: Increasing the accuracy of your model. Running the cell below will import the challenge dataset and will be available to you as `data`. Before running the cell below store the dataset in this file structure (or change the `data_path` accordingly):  

- `challenge_notebook.ipynb`
- `dataset`
    - `challenge_dataset_4_9.csv`


In [2]:
#data_path='./dataset/'
data_path='../'
data = np.loadtxt(data_path + "challenge_dataset_4_9_dim3_training.csv", delimiter=",")

#ind = np.random.permutation(data.shape[0])

# extracting the first column which contains the labels
data_labels = data[:, :1].reshape(data.shape[0],)
# extracting all the columns but the first which are our features
data_features = data[:, 1:]

## Visualizing the dataset

Before we dive into solving the question it is always beneficial to look at the dataset pictographically. This will help us understand patterns which we could leverage when designing our feature maps and variational circuits for example.

## Extracting the training dataset

The given dataset has already been reduced in dimension and normalized, so, further pre-processing isn't techincally required. You can do so if you want to, but the testing dataset will be of the same dimension and normalisation as the training dataset provided. Training a dataset of size 6,000 will take multiple hours so you'll need to extract a subset of the dataset to use as a training dataset. The accuracy of the model may vary based on the datapoints and size of the training dataset you choose. Thus, experimenting with various sizes and datapoints will be necessary. For example, Increasing the training dataset size may increase the accuracy of the model however it will increase the training time as well.

Use the space below to extract your training dataset from `data`. For your convenience `data` has been segregated into `data_labels` and `data_features`.

* `data_labels` : 6,000 $\times$ 1 column vector with each entry either `4` or `9` 
* `data_features` : 6,000 $\times$ 3 matrix with each row having the feature corresponding to the label in `data_labels`

**Note:** This process was done in the previous [VQC notebook](https://github.com/Qiskit-Challenge-India/2020/blob/master/Day%206%2C%207%2C8/VQC_notebook.ipynb) with `0` and `1` labels and can be modified and used here as well. 

In [3]:
### WRITE YOUR CODE BETWEEN THESE LINES - START

# do your classical pre-processing here

# store your training and testing datasets to be input in the VQC optimizer in the "training_input" and 
# "testing_input" variables respectively. These variables will eb accessed whiile creating a VQC instance later. 
four_datapoints = []
nine_datapoints = []

for i in range(6000):
    if data_labels[i] == 4:
        four_datapoints.append(data_features[i])

for i in range(6000):
    if data_labels[i] == 9:
        nine_datapoints.append(data_features[i])
        
four_datapoints = np.array(four_datapoints)
nine_datapoints = np.array(nine_datapoints)
np.random.shuffle(four_datapoints)
np.random.shuffle(nine_datapoints)

train_num = 200
test_num = 100

training_input = {4:four_datapoints[:train_num], 9:nine_datapoints[:train_num]}
test_input = {4:four_datapoints[-test_num:], 9:nine_datapoints[-test_num:]}

### WRITE YOUR CODE BETWEEN THESE LINES - END

## Building a Quantum Feature Map

Given below is the `feature_map()` function. It takes no input and has to return a feature map which is either a `FeatureMap` or `QuantumCircuit` object. In the previous notebook you've learnt how feature maps work and the process of using existing feature maps in Qiskit or creating your own. In the space given **inside the function** you have to create a feature map and return it.   


**IMPORTANT:** 
* If you require Qiskit import statements other than the ones provided in the cell below, please include them inside the appropriate space provided. **All additional import statements must be Qiskit imports.** 
* the first line of the cell below must be `%%write_and_run feature_map.py`. This function stores the content of the cell below in the file `feature_map.py`

In [4]:
%%write_and_run feature_map.py
# the write_and_run function writes the content in this cell into the file "feature_map.py"
    
# import libraries that are used in the function below.
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector

def feature_map(): 

    ckt9_depth=7
    num_qubits=3
    x = ParameterVector('x', length=3)
    feature_map=QuantumCircuit(num_qubits)
    
    for i in range(ckt9_depth):
        for j in range(num_qubits):
            feature_map.h(j)
        for j in range(2):
            feature_map.crz((np.pi-x[j]*x[j])*(np.pi-x[j+1]*x[j+1]),j,j+1)
        for j in range(3):
            feature_map.rx(x[j]*x[j],j)

    #return the feature map which is either a FeatureMap or QuantumCircuit object
    return feature_map

In [5]:
feature_map().draw()

┌───┐                                                                 »
q_0: ┤ H ├────────────────────────────────■────────────────────────────────»
     ├───┤┌───────────────────────────────┴───────────────────────────────┐»
q_1: ┤ H ├┤ RZ((3.14159265358979 - x[0]**2)*(3.14159265358979 - x[1]**2)) ├»
     ├───┤└───────────────────────────────────────────────────────────────┘»
q_2: ┤ H ├─────────────────────────────────────────────────────────────────»
     └───┘                                                                 »
«                              ┌─────────────┐                         »
«q_0: ─────────────────────────┤ RX(x[0]**2) ├─────────────────────────»
«                              └─────────────┘                         »
«q_1: ────────────────────────────────■────────────────────────────────»
«     ┌───────────────────────────────┴───────────────────────────────┐»
«q_2: ┤ RZ((3.14159265358979 - x[1]**2)*(3.14159265358979 - x[2]**2)) ├»
«     └───────────────────────────────────────────────────────────────┘»
«          ┌───┐          »
«q_0: ─────┤ H ├──────────»
«     ┌────┴───┴────┐┌───┐»
«q_1: ┤ RX(x[1]**2) ├┤ H ├»
«     ├─────────────┤├───┤»
«q_2: ┤ RX(x[2]**2) ├┤ H ├»
«     └─────────────┘└───┘»
«                                                                      »
«q_0: ────────────────────────────────■────────────────────────────────»
«     ┌───────────────────────────────┴───────────────────────────────┐»
«q_1: ┤ RZ((3.14159265358979 - x[0]**2)*(3.14159265358979 - x[1]**2)) ├»
«     └───────────────────────────────────────────────────────────────┘»
«q_2: ─────────────────────────────────────────────────────────────────»
«                                                                      »
«                              ┌─────────────┐                         »
«q_0: ─────────────────────────┤ RX(x[0]**2) ├─────────────────────────»
«                              └─────────────┘                         »
«q_1: ────────────────────────────────■────────────────────────────────»
«     ┌───────────────────────────────┴───────────────────────────────┐»
«q_2: ┤ RZ((3.14159265358979 - x[1]**2)*(3.14159265358979 - x[2]**2)) ├»
«     └───────────────────────────────────────────────────────────────┘»
«          ┌───┐          »
«q_0: ─────┤ H ├──────────»
«     ┌────┴───┴────┐┌───┐»
«q_1: ┤ RX(x[1]**2) ├┤ H ├»
«     ├─────────────┤├───┤»
«q_2: ┤ RX(x[2]**2) ├┤ H ├»
«     └─────────────┘└───┘»
«                                                                      »
«q_0: ────────────────────────────────■────────────────────────────────»
«     ┌───────────────────────────────┴───────────────────────────────┐»
«q_1: ┤ RZ((3.14159265358979 - x[0]**2)*(3.14159265358979 - x[1]**2)) ├»
«     └───────────────────────────────────────────────────────────────┘»
«q_2: ─────────────────────────────────────────────────────────────────»
«                                                                      »
«                              ┌─────────────┐                         »
«q_0: ─────────────────────────┤ RX(x[0]**2) ├─────────────────────────»
«                              └─────────────┘                         »
«q_1: ────────────────────────────────■────────────────────────────────»
«     ┌───────────────────────────────┴───────────────────────────────┐»
«q_2: ┤ RZ((3.14159265358979 - x[1]**2)*(3.14159265358979 - x[2]**2)) ├»
«     └───────────────────────────────────────────────────────────────┘»
«          ┌───┐          »
«q_0: ─────┤ H ├──────────»
«     ┌────┴───┴────┐┌───┐»
«q_1: ┤ RX(x[1]**2) ├┤ H ├»
«     ├─────────────┤├───┤»
«q_2: ┤ RX(x[2]**2) ├┤ H ├»
«     └─────────────┘└───┘»
«                                                                      »
«q_0: ────────────────────────────────■────────────────────────────────»
«     ┌───────────────────────────────┴───────────────────────────────┐»
«q_1: ┤ RZ((3.14159265358979 - x[0]**2)*(3.14159265358979 - x[1]**2)) ├»
«     └────────────────────────

## Building a Variational Circuit

Given below is the `variational_circuit()` function. It takes no input and has to return a variational circuit which is either a `VariationalForm` or `QuantumCircuit` object. In the previous notebook you've learnt how variational circuits work and the process of using existing variational circuit in Qiskit or creating your own. You have to create a variational circuit in the space given **inside the function** and return it. You can find various variational circuits in the [Qiskit Circuit Library](https://qiskit.org/documentation/apidoc/circuit_library.html) under N-local circuits.

**IMPORTANT:** 
* If you require Qiskit import statements other than the ones provided in the cell below, please include them inside the appropriate space provided. **All additional import statements must be Qiskit imports.** 
* the first line of the cell below must be `%%write_and_run feature_map.py`. This function stores the content of the cell below in the file `variational_circuit.py`

In [6]:
%%write_and_run variational_circuit.py
# the write_and_run function writes the content in this cell into the file "variational_circuit.py"

def variational_circuit():

    ckt5_depth=2
    num_qubits=3
    y = ParameterVector('y', length=num_qubits*(num_qubits+3)*ckt5_depth)
    var_circuit=QuantumCircuit(num_qubits)
    ii = 0
    
    for i in range(ckt5_depth):
        for j in range(num_qubits):
            var_circuit.rx(y[ii], j)
            ii += 1
        for j in range(num_qubits):
            var_circuit.rz(y[ii], j)
            ii += 1
        for j in range(num_qubits):
            for k in range(num_qubits):
                if j != k:
                    var_circuit.crz(y[ii], num_qubits-1-j, num_qubits-1-k)
                    ii += 1
        for j in range(num_qubits):
            var_circuit.rx(y[ii], j)
            ii += 1
        for j in range(num_qubits):
            var_circuit.rz(y[ii], j)
            ii += 1

    # return the variational circuit which is either a VaritionalForm or QuantumCircuit object
    return var_circuit

In [7]:
variational_circuit().draw()

┌──────────┐┌──────────┐            ┌──────────┐            ┌──────────┐»
q_0: ┤ RX(y[0]) ├┤ RZ(y[3]) ├────────────┤ RZ(y[7]) ├────────────┤ RZ(y[9]) ├»
     ├──────────┤├──────────┤┌──────────┐└────┬─────┘            └────┬─────┘»
q_1: ┤ RX(y[1]) ├┤ RZ(y[4]) ├┤ RZ(y[6]) ├─────┼───────────■───────────■──────»
     ├──────────┤├──────────┤└────┬─────┘     │      ┌────┴─────┐            »
q_2: ┤ RX(y[2]) ├┤ RZ(y[5]) ├─────■───────────■──────┤ RZ(y[8]) ├────────────»
     └──────────┘└──────────┘                        └──────────┘            »
«                               ┌───────────┐┌───────────┐┌───────────┐»
«q_0: ──────■────────────■──────┤ RX(y[12]) ├┤ RZ(y[15]) ├┤ RX(y[18]) ├»
«           │      ┌─────┴─────┐├───────────┤├───────────┤├───────────┤»
«q_1: ──────┼──────┤ RZ(y[11]) ├┤ RX(y[13]) ├┤ RZ(y[16]) ├┤ RX(y[19]) ├»
«     ┌─────┴─────┐├───────────┤├───────────┤├───────────┤├───────────┤»
«q_2: ┤ RZ(y[10]) ├┤ RX(y[14]) ├┤ RZ(y[17]) ├┤ RX(y[20]) ├┤ RZ(y[23]) ├»
«     └───────────┘└───────────┘└───────────┘└───────────┘└───────────┘»
«     ┌───────────┐             ┌───────────┐             ┌───────────┐»
«q_0: ┤ RZ(y[21]) ├─────────────┤ RZ(y[25]) ├─────────────┤ RZ(y[27]) ├»
«     ├───────────┤┌───────────┐└─────┬─────┘             └─────┬─────┘»
«q_1: ┤ RZ(y[22]) ├┤ RZ(y[24]) ├──────┼────────────■────────────■──────»
«     └───────────┘└─────┬─────┘      │      ┌─────┴─────┐             »
«q_2: ───────────────────■────────────■──────┤ RZ(y[26]) ├─────────────»
«                                            └───────────┘             »
«                               ┌───────────┐┌───────────┐
«q_0: ──────■────────────■──────┤ RX(y[30]) ├┤ RZ(y[33]) ├
«           │      ┌─────┴─────┐├───────────┤├───────────┤
«q_1: ──────┼──────┤ RZ(y[29]) ├┤ RX(y[31]) ├┤ RZ(y[34]) ├
«     ┌─────┴─────┐├───────────┤├───────────┤└───────────┘
«q_2: ┤ RZ(y[28]) ├┤ RX(y[32]) ├┤ RZ(y[35]) ├─────────────
«     └───────────┘└───────────┘└───────────┘

## Choosing a Classical Optimizer

In the `classical_optimizer()` function given below you will have to import the optimizer of your choice from [`qiskit.aqua.optimizers`](https://qiskit.org/documentation/apidoc/qiskit.aqua.components.optimizers.html) and return it. This function will not be called by the grading function `grade()` and thus the name of the function `classical_optimizer()`can be changed if needed. 

In [8]:
def classical_optimizer():
    # CHOOSE AND RETURN CLASSICAL OPTIMIZER OBJECT - START
    
    # import the required clasical optimizer from qiskit.aqua.optimizers
    from qiskit.aqua.components.optimizers import SPSA
    
    # create an optimizer object
    cls_opt = COBYLA(maxiter=500, tol=0.001)
    
    # CHOOSE AND RETURN CLASSICAL OPTIMIZER OBJECT - END
    return cls_opt

### Callback Function

The `VQC` class can take in a callback function to which the following parameters will be passed after every optimization cycle of the algorithm:

* `eval_count` : the evaulation counter
* `var_params` : value of parameters of the variational circuit
* `eval_val`  : current cross entropy cost 
* `index` : the batch index

In [9]:
def call_back_vqc(eval_count, var_params, eval_val, index):
    print("eval_count: {}".format(eval_count))
    print("var_params: {}".format(var_params))
    print("eval_val: {}".format(eval_val))
    print("index: {}".format(index))

## Optimization Step

This is where the whole VQC algorithm will come together. First we create an instance of the `VQC` class. 

In [10]:
# a fixed seed so that we get the same answer when the same input is given. 
seed = 1059

# setting our backend to qasm_simulator with the "statevector" method on. This particular setup is given as it was 
# found to perform better than most. Feel free to play around with different backend options.
backend = Aer.get_backend('qasm_simulator')
backend_options = {"method": "statevector"}

# creating a quantum instance using the backend and backend options taken before
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed, 
                                   backend_options=backend_options)
#quantum_instance = QuantumInstance(backend, shots=1024, backend_options=backend_options)

# creating a VQC instance which you will be used for training. Make sure you input the correct training_dataset and 
# testing_dataset as defined in your program.
vqc = VQC(optimizer=classical_optimizer(), 
          feature_map=feature_map(), 
          var_form=variational_circuit(), 
          callback=call_back_vqc, 
          training_dataset=training_input,     # training_input must be initialized with your training dataset
          test_dataset=test_input)             # testing_input must be initialized with your testing dataset

Now, let's run the VQC classification routine

In [ ]:
start = time.process_time()

result = vqc.run(quantum_instance)

print("time taken: ")
print(time.process_time() - start)

print("testing success ratio: {}".format(result['testing_accuracy']))

eval_count: 0
var_params: [ 0.6207657  -1.20757153 -0.78879466  1.88707524  0.32880214  1.49412602
  0.54116455 -1.37485648  0.38284034 -0.50527287 -0.90656154  1.57967323
  0.50957082  1.11747093 -1.20613209 -0.54523593  0.61485565  0.16128901
  0.20578956  0.26484438 -0.42862367  0.30071293 -0.34538476  1.33053407
  0.72620856  1.63210013  1.11602692 -0.77856674  2.59368918  0.22011001
  0.38793627 -0.39438313  0.38235928  1.59918627  1.86063055  1.47123423]
eval_val: 0.6235178746998697
index: 0
eval_count: 1
var_params: [ 1.6207657  -1.20757153 -0.78879466  1.88707524  0.32880214  1.49412602
  0.54116455 -1.37485648  0.38284034 -0.50527287 -0.90656154  1.57967323
  0.50957082  1.11747093 -1.20613209 -0.54523593  0.61485565  0.16128901
  0.20578956  0.26484438 -0.42862367  0.30071293 -0.34538476  1.33053407
  0.72620856  1.63210013  1.11602692 -0.77856674  2.59368918  0.22011001
  0.38793627 -0.39438313  0.38235928  1.59918627  1.86063055  1.47123423]
eval_val: 0.607641199523167
inde

eval_count: 17
var_params: [ 1.6207657  -1.20757153 -0.78879466  2.88707524  0.32880214  1.49412602
  0.54116455 -1.37485648  0.38284034  0.49472713 -0.90656154  1.57967323
  0.50957082  1.11747093 -1.20613209 -0.54523593  1.61485565  0.16128901
  0.20578956  0.26484438 -0.42862367  0.30071293 -0.34538476  1.33053407
  0.72620856  1.63210013  1.11602692 -0.77856674  2.59368918  0.22011001
  0.38793627 -0.39438313  0.38235928  1.59918627  1.86063055  1.47123423]
eval_val: 0.6085132836352816
index: 17
eval_count: 18
var_params: [ 1.6207657  -1.20757153 -0.78879466  2.88707524  0.32880214  1.49412602
  0.54116455 -1.37485648  0.38284034  0.49472713 -0.90656154  1.57967323
  0.50957082  1.11747093 -1.20613209 -0.54523593  0.61485565  1.16128901
  0.20578956  0.26484438 -0.42862367  0.30071293 -0.34538476  1.33053407
  0.72620856  1.63210013  1.11602692 -0.77856674  2.59368918  0.22011001
  0.38793627 -0.39438313  0.38235928  1.59918627  1.86063055  1.47123423]
eval_val: 0.6014157441778204


eval_count: 34
var_params: [ 1.6207657  -1.20757153 -0.78879466  2.88707524  0.32880214  1.49412602
  0.54116455 -1.37485648  0.38284034  0.49472713 -0.90656154  1.57967323
  0.50957082  1.11747093 -1.20613209 -0.54523593  0.61485565  0.16128901
  0.20578956  1.26484438 -0.42862367  0.30071293 -0.34538476  1.33053407
  0.72620856  1.63210013  1.11602692 -0.77856674  2.59368918  0.22011001
  0.38793627 -0.39438313  1.38235928  2.59918627  1.86063055  1.47123423]
eval_val: 0.6175350575602656
index: 34
eval_count: 35
var_params: [ 1.6207657  -1.20757153 -0.78879466  2.88707524  0.32880214  1.49412602
  0.54116455 -1.37485648  0.38284034  0.49472713 -0.90656154  1.57967323
  0.50957082  1.11747093 -1.20613209 -0.54523593  0.61485565  0.16128901
  0.20578956  1.26484438 -0.42862367  0.30071293 -0.34538476  1.33053407
  0.72620856  1.63210013  1.11602692 -0.77856674  2.59368918  0.22011001
  0.38793627 -0.39438313  1.38235928  1.59918627  2.86063055  1.47123423]
eval_val: 0.5781677579484137


eval_count: 51
var_params: [ 2.20987687 -1.23641173 -0.78701373  2.96209936  0.34683867  1.53167052
  0.45113659 -1.3595667   0.36643168  0.83349692 -0.72333983  1.34763455
  0.64870994  1.21797227 -1.35793748 -0.47165845  0.72203625  0.28337531
  0.3221959   1.1460753  -0.21958453  0.50723199 -0.19223417  1.56691504
  0.87689081  1.89172053  1.38968978 -0.50490389  2.86735204  0.42815166
  0.62084854 -0.15808482  1.6589974   1.52955098  2.87367146  1.46048159]
eval_val: 0.5572431712522568
index: 51
eval_count: 52
var_params: [ 2.25849821 -1.25057079 -0.78613938  2.99893235  0.30702824  1.55010295
  0.40693747 -1.3520602   0.35837587  0.83010149 -0.54970874  1.32664334
  0.57299454  1.1232877  -0.88573222 -0.57956125  0.63063085  0.19928786
  0.23531986  1.05886476 -0.23208159  0.4934977  -0.23216965  1.56784141
  0.83574351  1.90405627  1.40891964 -0.48567402  2.8865819   0.41516487
  0.62007196 -0.15719903  1.67968796  1.49536363  2.88007388  1.4552026 ]
eval_val: 0.5521657102751519


eval_count: 68
var_params: [ 2.22881796 -1.24192758 -0.78667311  2.97644815  0.33132998  1.53885112
  0.43391823 -1.35664244  0.36329343  0.83217418 -0.88241005  1.33945714
  0.47826094  1.21200329 -1.15291749 -0.48121171  1.11808548  0.10685231
  0.41465926  0.98110147 -0.24100333  0.49094994 -0.20941125  1.37884145
  0.85917156  2.34643211  1.08147202 -0.56140887  2.81084705  0.41345653
  0.61061195  0.00784815  1.60306145  1.51623284  2.87616561  1.45842509]
eval_val: 0.5432897654605253
index: 68
eval_count: 69
var_params: [ 2.21900398 -1.23906964 -0.7868496   2.96901359  0.33936552  1.53513063
  0.4428396  -1.35815759  0.36491946  0.83285954 -0.88049078  1.34369412
  0.47677468  1.21211169 -1.15796771 -0.47924619  1.11371075  0.10986846
  0.41279058  0.98324806 -0.23319632  0.49338795 -0.21099665  1.37883846
  0.85790084  1.89734466  1.16244294 -0.64817552  3.03022005  0.41444794
  0.6094369   0.00738403  1.82213961  1.52749346  2.87487331  1.45949063]
eval_val: 0.5370940995483187


...

eval_count: 134
var_params: [ 2.22431925 -1.23428202 -0.7952263   3.06611716  0.53028225  1.57292533
  0.47930269 -1.29883143  0.37727509  1.01258031 -0.90438655  1.57208943
  0.36233759  1.00965269 -1.04636793 -0.50111498  1.09935254  0.02294684
  0.32276753  1.00240719 -0.62134275  0.50091571 -0.37903028  1.29355986
  0.90229268  1.73949166  1.01621298 -0.97468389  2.59866359  0.27972994
  0.47143898  0.04873141  1.87991702  1.56780978  2.76658058  1.48348596]
eval_val: 0.5262779948462297
index: 134
eval_count: 135
var_params: [ 2.21454710e+00 -1.24676396e+00 -8.02766285e-01  3.08045131e+00
  5.34143498e-01  1.58248431e+00  4.81445567e-01 -1.28567085e+00
  3.79019213e-01  1.02835129e+00 -9.03439247e-01  1.57986868e+00
  3.45647923e-01  1.24154964e+00 -1.02869970e+00 -6.11199462e-01
  1.06399646e+00 -1.61606485e-03  2.92464403e-01  9.86102962e-01
 -6.50036208e-01  4.98735773e-01 -3.87297284e-01  1.33171656e+00
  9.08954588e-01  1.66771225e+00  1.01654100e+00 -9.69505364e-01
  2.607420

eval_count: 150
var_params: [ 2.21635348 -1.27358542 -0.79019444  3.09228778  0.5984822   1.52833641
  0.53243957 -1.27402252  0.3465971   0.98967675 -0.92231818  1.53315446
  0.37725079  1.06033859 -1.02552777 -0.56782361  1.12187686 -0.01586133
  0.21300235  1.03596234 -0.64292227  0.57841482 -0.41710989  1.30483273
  0.86341511  1.84409012  1.06565741 -0.93271998  2.56443152  0.18712126
  0.47665987  0.02505784  1.85261496  1.57303496  2.72656908  1.52979919]
eval_val: 0.5247591840399594
index: 150
eval_count: 151
var_params: [ 2.2259504  -1.30589157 -0.79043483  3.08132088  0.4480893   1.48165425
  0.50333693 -1.40158401  0.34237089  0.98885868 -0.92821063  1.56079346
  0.37661656  1.08790168 -1.06699429 -0.54904139  1.10514036  0.01120554
  0.21246565  1.04857959 -0.60889206  0.58020033 -0.4172616   1.29748035
  0.83798732  1.82181971  1.11868967 -0.88102796  2.50378436  0.19173242
  0.45957417  0.03072662  1.77129917  1.56589509  2.7979418   1.44369626]
eval_val: 0.52448197203774

eval_count: 166
var_params: [ 2.21867814 -1.26826622 -0.79779099  3.05701595  0.52956954  1.4844137
  0.53444456 -1.30388927  0.36156715  0.92207318 -0.91736876  1.50997773
  0.39866302  1.08162588 -1.06290976 -0.57278187  1.12350262 -0.0085245
  0.22631472  1.04297229 -0.60573241  0.57045671 -0.37972991  1.33951209
  0.8262537   1.84155568  1.08160645 -1.00181217  2.61602893  0.22714841
  0.50447307  0.02809587  1.87825229  1.57870488  2.77847456  1.52143882]
eval_val: 0.5256476109538778
index: 166
eval_count: 167
var_params: [ 2.21436705 -1.28166909 -0.7855981   3.08265435  0.54276113  1.50671643
  0.5262614  -1.32262553  0.33918211  0.99501082 -0.90268323  1.53940699
  0.39407725  1.06961348 -1.06916931 -0.56364962  1.08155542  0.01314234
  0.21133266  1.05810334 -0.63882856  0.57573543 -0.40576209  1.30683965
  0.85567872  1.84138029  1.07377675 -1.00974043  2.60802033  0.19220657
  0.47176122  0.0293063   1.89104496  1.57284204  2.75472642  1.49720014]
eval_val: 0.5237064189403399

eval_count: 183
var_params: [ 2.25033133 -1.29431819 -0.78633246  3.02809525  0.52879005  1.48803282
  0.55805427 -1.3227983   0.31226738  1.02235061 -0.91972259  1.62387388
  0.39085105  1.03265126 -1.05850715 -0.65296023  1.07185658 -0.02121414
  0.16036442  1.0721772  -0.6324962   0.61743822 -0.46265194  1.29039635
  0.85794376  1.88826314  1.06087297 -1.03000576  2.58363556  0.21633043
  0.51459422  0.0353577   1.91883404  1.56139534  2.7682635   1.48581155]
eval_val: 0.5223732427724963
index: 183
eval_count: 184
var_params: [ 2.26666513 -1.33293119 -0.8182637   3.01456732  0.52790711  1.49557861
  0.54349853 -1.3232401   0.31103933  1.02038453 -0.91796608  1.61467208
  0.37037442  0.98247857 -1.05814243 -0.62925872  1.0468389  -0.02582789
  0.19227344  1.0384029  -0.61161933  0.64305064 -0.43458493  1.30225856
  0.93451297  1.86993654  1.05946979 -1.03116179  2.59451916  0.22960959
  0.49337891  0.0710325   1.91926404  1.55884817  2.77845449  1.4856734 ]
eval_val: 0.52201693419546

eval_count: 200
var_params: [ 2.3068624  -1.16649985 -0.81259097  3.13320104  0.47758367  1.44445839
  0.61328217 -1.39214751  0.32221759  1.06572248 -0.89313077  1.57781392
  0.39991279  0.96029816 -1.09818026 -0.64931137  1.08948181 -0.06901653
  0.14750387  1.03032501 -0.63998175  0.59936439 -0.46468226  1.2825212
  0.93596243  1.87103197  1.01805587 -1.06982276  2.54361466  0.25912184
  0.49765074  0.04360592  1.9280173   1.53896817  2.71371879  1.50123057]
eval_val: 0.5226078510035047
index: 200
eval_count: 201
var_params: [ 2.29960937 -1.21438785 -0.74456813  3.11258025  0.49222269  1.43281704
  0.62438705 -1.37107525  0.3348869   1.01946329 -0.87363049  1.6142093
  0.40359982  0.97878871 -1.08582104 -0.62768995  1.09133598 -0.0537799
  0.16925318  1.03600162 -0.62838065  0.61791718 -0.44524325  1.27538153
  0.93794794  1.86675912  1.02097154 -1.06633761  2.52222593  0.25887024
  0.50512239  0.04602821  1.94994867  1.54541836  2.67606623  1.45558014]
eval_val: 0.5209923487016833


In [16]:
print(result)

<module 'qiskit.result' from '/home/nimay/anaconda3/lib/python3.7/site-packages/qiskit/result/__init__.py'>


## Storing the optimal parameters for grading

Once the training step of the vqc algorithm is done we obtain the optimal parameters for our specific variational form. For the grading function to be able to access these optimal parameters you will need to follow the steps below. 

* **Step 1**: Run the cell below with `print(repr(vqc.optimal_params))`. 
* **Step 2**: Copy the matrix of optimal parameters and store it in the variable `optimal_parameters` inside the function `return_optimal_params()` in the next cell. This will enable us to extract it while calculating the accuracy your the model during grading. Given below is a pictographical explanation of the same:  

<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/opt_params456b075.png" width="800">

In [217]:
print(repr(vqc.optimal_params))

array([ 0.54632732,  0.40538044, -1.01295329,  1.49487678,  1.64501921,
        0.59286665,  2.12784723,  1.21054489, -0.5643853 ,  0.95213542,
       -0.27556139, -1.83490042, -0.01200562,  3.75627203,  1.43557252,
       -0.39997671, -0.5545942 ,  0.46746988, -1.27218015,  0.0318519 ,
        0.87543005, -0.25437162, -1.34021757, -2.08737164,  2.04253738,
        0.83175021,  0.77956833, -0.17427723,  2.16326879, -1.5828532 ,
        1.17114289, -1.17920076, -1.45410869, -0.89369175,  0.01718402,
       -1.59692096,  0.34737521, -2.48543596,  0.74288619,  0.28864511,
        0.57559697,  0.51204554, -0.92355601,  0.49001691, -0.6171768 ,
        0.14289195,  1.87852521,  0.44203041,  2.02773329,  1.01900935,
        1.40938457, -0.3939512 ,  0.50795543,  0.43727053,  0.64402482,
        0.43716062, -2.43465227,  0.81417097,  0.38247537, -0.64960273,
       -0.71656997,  0.64017647,  0.91821865, -0.04276097, -2.2802225 ,
       -0.90311979,  1.9641994 , -0.9093311 , -0.08099036,  0.13

In [12]:
%%write_and_run optimal_params.py
# # the write_and_run function writes the content in this cell into the file "optimal_params.py"

### WRITE YOUR CODE BETWEEN THESE LINES - START
    
# import libraries that are used in the function below.
import numpy as np
    
### WRITE YOUR CODE BETWEEN THESE LINES - END

def return_optimal_params():
    # STORE THE OPTIMAL PARAMETERS AS AN ARRAY IN THE VARIABLE optimal_parameters 
    
    optimal_parameters = [2.29996024,-1.2277486,-0.76346192,3.14999219,0.50483292,1.45148143
,0.67203373,-1.37036151,0.2957781,1.02428077,-0.91873262,1.67605111
,0.33861149,1.03158828,-1.08958088,-0.58049943,1.07004413,-0.08628848
,0.2291223,1.09280666,-0.65799625,0.64149389,-0.46584535,1.29714015
,0.94446357,1.87499855,1.00179137,-1.07566,2.49502226,0.29832526
,0.47791824,0.05945576,1.9662437,1.54163725,2.66970775,1.48642167]
    
    # STORE THE OPTIMAL PARAMETERS AS AN ARRAY IN THE VARIABLE optimal_parameters 
    return np.array(optimal_parameters)

## Submission

Before we go any further, check that you have the three files `feature_map.py`, `variational_circuit.py` and `optimal_params.py` in the **same working directory as this notebook**. If you do not, then go back to the start and run the notebook making sure you have filled in the code where its required. When you run the cell below, all the three files `feature_map.py`, `variational_circuit.py` and `optimal_params.py` are combined into one file named **"answer.py"**. Now your working directory will have four python (.py) files out of which **"answer.py"** is the submission file: 
* `answer.py` <- upload this file onto HackerEarth and click on "Submit and Evaluate"
* `feature_map.py`
* `variational_circuit.py`
* `optimal_params.py`

In [13]:
solution = ['feature_map.py','variational_circuit.py','optimal_params.py']
file = open("answer.py","w")
file.truncate(0)
for i in solution:    
    with open(i) as f:
        with open("answer.py", "a") as f1:
            for line in f:
                f1.write(line)
file.close()

## Grading Function

Given below is the grading function that we shall use to grade your submission with a test dataset that is of the same format as `challenge_dataset_4_9.csv`. You can use it to grade your submission by extracting a few points out of the `challenge_dataset_4_9.csv` to get a basic idea of how your model is performing. 

In [14]:
#imports required for the grading function 
from qiskit import *
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.feature_maps import FeatureMap
from qiskit.aqua.components.variational_forms import VariationalForm
import numpy as np

### Working of the grading function

The grading function `grade()` takes as **input**: 

* `test_data`: (`np.ndarray`) -- **no. of datapoints $\times$ dimension of data** : the datapoints against which we want to test our model. 


* `test_labels`: (`np.ndarray`) -- **no. of datapoints $\times$ 1** : A column vector with each entry either 0 or 1 as entries.


* `feature_map`: (`QuantumCircuit` or `FeatureMap`) -- A quantum feature map which is the output of `feature_map()` defined earlier.


* `variational_form`: (`QuantumCircuit` or `VariationalForm`) -- A variational form which is the output of `variational_circuit()` defined earlier.


* `optimal_params`: (`numpy.ndarray`) -- the optimal parameters obtained after running the VQC algorithm above. These are the values obtained when the function `return_optimal_params()` is run. 


* `find_circuit_cost` : (`bool`) -- Calculates the circuit cost if set to `True`. Circuit cost is calculated by converting the circuit to the basis gate set `\[ 'u3', 'cx'\]` and then applying the formula **cost = 1$\times$(no.of u3 gates) + 10$\times$(no.of cx gates)**.


* `verbose` : (`bool`) -- prints the result message if set to `True`.

And gives as **output**: 

* `model_accuracy` : (`numpy.float64`) -- percent accuracy of the model. 


* `circuit_cost`: (`int`) -- circuit cost as explained above.


* `ans`: (`tuple`) -- Output of the `VQC.predict()` method. 


* `result_msg`: (`str`) -- Result message which also outputs the error message in case of one.


* `unrolled_circuit`: (`QuantumCircuit` or `None`) -- the circuit obtained after unrolling the full VQC circuit and substituting the optimal parameters to the basis gate set `\[ 'u3', 'cx'\]`.

**Note:** if you look inside the `grade()` function in Section 2 you'll see that we have initialized a COBYLA optimizer though the prediction step will not require one. Similarily we have given a dataset to `training dataset`. Both of these are dummy variables. The reason for this is because these are not optional variables the `VQC` class instantiation.  

In [15]:
def grade(test_data, test_labels, feature_map, variational_form, optimal_params, find_circuit_cost=True, verbose=True):
    seed = 10598
    model_accuracy = None 
    circuit_cost=None 
    ans = None
    unrolled_circuit = None
    result_msg=''
    data_dim = np.array(test_data).shape[1]
    dataset_size = np.array(test_data).shape[0]
    dummy_training_dataset=training_input = {'A':np.ones((2,data_dim)), 'B':np.ones((2, data_dim))}
    
    # converting 4's to 0's and 9's to 1's for checking 
    test_labels_transformed = np.where(test_labels==4, 0., 1.)
    max_qubit_count = 6
    max_circuit_cost = 2000
    
    # Section 1
    if feature_map is None:
        result_msg += 'feature_map variable is None. Please submit a valid entry' if verbose else ''
    elif variational_form is None: 
        result_msg += 'variational_form variable is None. Please submit a valid entry' if verbose else ''
    elif optimal_params is None: 
        result_msg += 'optimal_params variable is None. Please submit a valid entry' if verbose else ''
    elif test_data is None: 
        result_msg += 'test_data variable is None. Please submit a valid entry' if verbose else ''
    elif test_labels is None: 
        result_msg += 'test_labels variable is None. Please submit a valid entry' if verbose else ''
    elif not isinstance(feature_map, (QuantumCircuit, FeatureMap)):
        result_msg += 'feature_map variable should be a QuantumCircuit or a FeatureMap not (%s)' % \
                      type(feature_map) if verbose else ''
    elif not isinstance(variational_form, (QuantumCircuit, VariationalForm)):
        result_msg += 'variational_form variable should be a QuantumCircuit or a VariationalForm not (%s)' % \
                      type(variational_form) if verbose else ''
    elif not isinstance(test_data, np.ndarray):
        result_msg += 'test_data variable should be a numpy.ndarray not (%s)' % \
                      type(test_data) if verbose else ''
    elif not isinstance(test_labels, np.ndarray):
        result_msg += 'test_labels variable should be a numpy.ndarray not (%s)' % \
                      type(test_labels) if verbose else ''
    elif not isinstance(optimal_params, np.ndarray):
        result_msg += 'optimal_params variable should be a numpy.ndarray not (%s)' % \
                      type(optimal_params) if verbose else ''
    elif not dataset_size == test_labels_transformed.shape[0]:
        result_msg += 'Dataset size and label array size must be equal'
    # Section 2
    else:
        
        # setting up COBYLA optimizer as a dummy optimizer
        from qiskit.aqua.components.optimizers import COBYLA
        dummy_optimizer = COBYLA()

        # setting up the backend and creating a quantum instance
        backend = Aer.get_backend('qasm_simulator')
        backend_options = {"method": "statevector"}
        quantum_instance = QuantumInstance(backend, 
                                           shots=2000, 
                                           seed_simulator=seed, 
                                           seed_transpiler=seed, 
                                           backend_options=backend_options)

        # creating a VQC instance and running the VQC.predict method to get the accuracy of the model 
        vqc = VQC(optimizer=dummy_optimizer, 
                  feature_map=feature_map, 
                  var_form=variational_form, 
                  training_dataset=dummy_training_dataset)
        
        from qiskit.transpiler import PassManager
        from qiskit.transpiler.passes import Unroller
        pass_ = Unroller(['u3', 'cx'])
        pm = PassManager(pass_)
        # construct circuit with first datapoint
        circuit = vqc.construct_circuit(data[0], optimal_params)
        unrolled_circuit = pm.run(circuit)
        gates = unrolled_circuit.count_ops()
        if 'u3' in gates: 
            circuit_cost = gates['u3']
        if 'cx' in gates: 
            circuit_cost+= 10*gates['cx']
        
        if circuit.num_qubits > max_qubit_count:
            result_msg += 'Your quantum circuit is using more than 6 qubits. Reduce the number of qubits used and try again.'
        elif circuit_cost > max_circuit_cost:
            result_msg += 'The cost of your circuit is exceeding the maximum accpetable cost of 2000. Reduce the circuit cost and try again.'
        else: 
            
            ans = vqc.predict(test_data, quantum_instance=quantum_instance, params=np.array(optimal_params))
            model_accuracy = np.sum(np.equal(test_labels_transformed, ans[1]))/len(ans[1])

            result_msg += 'Accuracy of the model is {}'.format(model_accuracy) if verbose else ''
            result_msg += ' and circuit cost is {}'.format(circuit_cost) if verbose else ''
            
    return model_accuracy, circuit_cost, ans, result_msg, unrolled_circuit

## Process of grading using a dummy grading dataset

Let us create a dummy grading dataset with features and labels `grading_features` and `grading_labels` created from the last 2000 datapoints from `data_features` and `data_labels`so that we can a rough estimate of our accuaracy. It must be noted that this may not be a balanced dataset, i.e, may not have equal number of `4`'s and `9`'s and is not best practice. This is only given for the purpose of the demo of `grade()` function. In the final scoring done on HackerEarth, the testing dataset used will have a balanced number of class labels `4` and `9`.

In [16]:
grading_dataset_size=2000    # this value is not per digit but in total
grading_features = data_features[-grading_dataset_size:]
grading_labels = data_labels[-grading_dataset_size:]

In [17]:
start = time.process_time()

accuracy, circuit_cost, ans, result_msg, full_circuit  =  grade(test_data=grading_features, 
                                                                test_labels=grading_labels, 
                                                                feature_map=feature_map(), 
                                                                variational_form=variational_circuit(), 
                                                                optimal_params=return_optimal_params())

print("time taken: {} seconds".format(time.process_time() - start))
print(result_msg)

time taken: 153.53146194200002 seconds
Accuracy of the model is 0.748 and circuit cost is 638


You can also check your **accuracy**, **circuit_cost** and **full_circuit** which is the result of combining the feature map and variational circuit and unrolling into the basis \['u3', 'cx'\].

In [52]:
print("Accuracy of the model: {}".format(accuracy))
print("Circuit Cost: {}".format(circuit_cost))
print("The complete unrolled circuit: ")
full_circuit.draw()

Accuracy of the model: 0.6835
Circuit Cost: 648
The complete unrolled circuit: 


┌───────────────┐                                                 »
q_0: ┤ U3(pi/2,0,pi) ├──────────────────────■───────────────────────■──»
     ├───────────────┤┌──────────────────┐┌─┴─┐┌─────────────────┐┌─┴─┐»
q_1: ┤ U3(pi/2,0,pi) ├┤ U3(0,0,-0.99523) ├┤ X ├┤ U3(0,0,0.99523) ├┤ X ├»
     ├───────────────┤└┬───────────────┬─┘└───┘└─────────────────┘└───┘»
q_2: ┤ U3(pi/2,0,pi) ├─┤ U3(0,0,2.597) ├───────────────────────────────»
     └───────────────┘ └───────────────┘                               »
c_0: ══════════════════════════════════════════════════════════════════»
                                                                       »
c_1: ══════════════════════════════════════════════════════════════════»
                                                                       »
c_2: ══════════════════════════════════════════════════════════════════»
                                                                       »
«     ┌─────────────────┐                                                ┌───┐»
«q_0: ┤ U3(0,0,-0.9614) ├────────────────────────────────────────────────┤ X ├»
«     └─────────────────┘                       ┌───────────────────────┐└─┬─┘»
«q_1: ─────────■─────────────────────────────■──┤ U3(0.8228,-pi/2,pi/2) ├──┼──»
«            ┌─┴─┐       ┌────────────────┐┌─┴─┐└───────────────────────┘  │  »
«q_2: ───────┤ X ├───────┤ U3(0,0,-2.597) ├┤ X ├───────────────────────────■──»
«            └───┘       └────────────────┘└───┘                              »
«c_0: ════════════════════════════════════════════════════════════════════════»
«                                                                             »
«c_1: ════════════════════════════════════════════════════════════════════════»
«                                                                             »
«c_2: ════════════════════════════════════════════════════════════════════════»
«                                                                             »
«     ┌────────────────┐                    ┌───┐   ┌──────────────────┐   »
«q_0: ┤ U3(0,0,0.9614) ├────────────────────┤ X ├───┤ U3(4,-pi/2,pi/2) ├───»
«     ├───────────────┬┘┌──────────────────┐└─┬─┘   └──────────────────┘   »
«q_1: ┤ U3(pi/2,0,pi) ├─┤ U3(0,0,-0.99523) ├──┼────────────────────────────»
«     └───────────────┘ └──────────────────┘  │  ┌────────────────────────┐»
«q_2: ────────────────────────────────────────■──┤ U3(0.90163,-pi/2,pi/2) ├»
«                                                └────────────────────────┘»
«c_0: ═════════════════════════════════════════════════════════════════════»
«                                                                          »
«c_1: ═════════════════════════════════════════════════════════════════════»
«                                                                          »
«c_2: ═════════════════════════════════════════════════════════════════════»
«                                                                          »
«     ┌───────────────┐                                         »
«q_0: ┤ U3(pi/2,0,pi) ├────────■─────────────────────────────■──»
«     └───────────────┘      ┌─┴─┐      ┌─────────────────┐┌─┴─┐»
«q_1: ───────────────────────┤ X ├──────┤ U3(0,0,0.99523) ├┤ X ├»
«     ┌───────────────┐┌─────┴───┴─────┐└─────────────────┘└───┘»
«q_2: ┤ U3(pi/2,0,pi) ├┤ U3(0,0,2.597) ├────────────────────────»
«     └───────────────┘└───────────────┘                        »
«c_0: ══════════════════════════════════════════════════════════»
«                                                               »
«c_1: ══════════════════════════════════════════════════════════»
«                                                               »
«c_2: ══════════════════════════════════════════════════════════»
«                                                               »
«     ┌─────────────────┐                                                ┌───┐»
«q_0: ┤ U3(0,0,-0.9614) ├────────────────────────────────────────────────┤ X 